## MNIST *.npz 파일 읽어서 처리

In [1]:
import numpy as np
import tensorflow as tf
from tqdm import trange

In [2]:
train = np.load('train.npz')
train_input = train['data']
train_label = train['target']

In [3]:
np.shape(train_input)

(42000, 784)

In [4]:
np.shape(train_label)

(42000, 10)

In [5]:
test = np.load('test.npz')
test_input = test["data"]
test_label = test['target']

In [6]:
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [7]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [8]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)

L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
L2_flat = tf.reshape(L2, [-1, 7*7*64])
W3 = tf.get_variable("W3", shape=[7*7*64, 10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [12]:
def shuffle_data(x_train, y_train):
    temp_index = np.arange(len(x_train))
    np.random.shuffle(temp_index)
    
    x_temp = np.zeros(x_train.shape)
    y_temp = np.zeros(y_train.shape)
    
    x_temp = x_train[temp_index]
    y_temp = y_train[temp_index]
    
    return x_temp, y_temp

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
epochs = 15
batch_size = 100
total_batch = int(len(train_input) / batch_size)

In [16]:
### 학습

for epoch in trange(epochs):
    avg_cost = 0
    train_input, train_label = shuffle_data(train_input, train_label)
    for i in range(total_batch):
        start = ((i+1) * batch_size) - batch_size
        end = ((i+1) * batch_size)
        
        batch_xs = train_input[start:end]
        batch_ys = train_label[start:end]
        
        feed_dict= {X:batch_xs, Y:batch_ys}
        _, cost_val = sess.run([optimizer, cost], feed_dict=feed_dict)
        avg_cost += cost_val / total_batch
    print('Epoch: %04d' % (epoch+1), ",cost : {:.4f}".format(avg_cost))
print("훈련 완료!!!")


  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

Epoch: 0001 ,cost : 0.0216



  7%|█████▌                                                                             | 1/15 [00:44<10:17, 44.08s/it]

Epoch: 0002 ,cost : 0.0183



 13%|███████████                                                                        | 2/15 [01:26<09:25, 43.47s/it]

...

Epoch: 0014 ,cost : 0.0091



 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [09:52<00:39, 39.20s/it]

Epoch: 0015 ,cost : 0.0099



100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [10:29<00:00, 38.42s/it]


훈련 완료!!!


In [18]:
## 테스트 정확도 확인
is_correct = tf.equal(tf.argmax(logits,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도 :", sess.run(accuracy, feed_dict={X:test_input, Y:test_label}))

정확도 : 0.985
